## 🌿Machine Learning Code Process
1. Import Package
2. Data Preparation --> Define input & output
<br>preprocessing(raw data, max-length, channel(image), ...)
<br>return dataframe
3. Model, Class
<br> Init(input_size=2)
<br>return longTensor().size() // (3, 3, 7) (batch, output, class)
4. Loss, Optim, Hyperparameter
5. Training Function(model, data, loss function, optimizer, ...)
6. Validation 
7. Prediction
8. Analysis



---



# 입력 문자열에 따른 다음 문자열 예측 모델 만들기
- 예제) hello --> ello
    - h --> e
    - e --> l
    - l --> l
    - o --> ?

## 0. Install Packages

In [2]:
import torch
import torch.optim as optim
import numpy as np
import pandas as pd

 ## 1. Data Prepreation

In [30]:
# 입력 데이터
sample = " if you want you"
type(sample) # str

str

In [31]:
# 입력 데이터 char 단위로 자르기
char_set = list(set(sample))    # char 단위로 sample 자르기 (dict -> list)
char_set

['y', ' ', 'a', 'n', 't', 'o', 'u', 'i', 'w', 'f']

In [32]:
# 입력 데이터 인덱스 만들기
char_dic = {char:idx for idx, char in enumerate(char_set)}
char_dic

{' ': 1,
 'a': 2,
 'f': 9,
 'i': 7,
 'n': 3,
 'o': 5,
 't': 4,
 'u': 6,
 'w': 8,
 'y': 0}

In [33]:
# hyperparameter 정의
dic_size = len(char_dic) # 10
hidden_size = len(char_dic) # 10
learning_rate = 0.1

In [ ]:
# index로 바꾸기
sample_idx = [char_dic[c] for c in sample]  # sample char를 index로 바꿈
sample_idx  # [1, 7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]

x_data = [sample_idx[:-1]] # 마지막만 제외하고 가져옴
x_data # [[1, 7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5]]

x_one_hot = [np.eye(dic_size)[x] for x in x_data]   # np.eye : one-hot encoding
x_one_hot # [array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],

y_data = [sample_idx[1:]] # 처음만 제외하고 가져옴
y_data # [[7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]]

In [ ]:
# 형변환 (pytorch long이나 float 형태만 연산이 가능)
X = torch.FloatTensor(x_one_hot)   # x_one_hot 형변환
X # tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],

Y = torch.LongTensor(y_data) # y_data 형변환
Y # tensor([[7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]])

## 3. Model Implimentation
* 참고 : https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

```
rnn = torch.nn.RNN(10, 20, 2, batch_first=True)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)
```

- RNN(input_size, hidden_size)
- input_size : 하나의 x 값이 몇 차원인지?
- sequence_size : 한 번에 넣는 수

In [94]:
# RNN model
input_size = 10   # 10
hidden_size = 10
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
input = torch.randn(1, 5, input_size) # input : batch_size, sequence_size, dimension = (1, 5, 10)
output, h0 = rnn(input)               # output: batch_size, seqeunce_size, hidden_size = (1, 5, 20)

In [95]:
# loss
criterion = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(rnn.parameters(), lr=0.001)

In [96]:
# declare RNN
rnn.train() # train mode
optimizer.zero_grad() # initialize gradient
output, _ = rnn(X) # input: X
print(output.size()) # torch.Size([1, 15, 20])
print(X.size())      # torch.Size([1, 15, 10])

torch.Size([1, 15, 10])
torch.Size([1, 15, 10])


In [104]:
# start training
for i in range(200):
    rnn.train()                         # train mode
    optimizer.zero_grad()               # initialize gradient
    output, _ = rnn(X)                  # input: X
    # print(output.size())                # torch.Size([1, 15, 10])
    # print(Y.size())                     # torch.Size([1, 15])
    # print(output.view(-1, 10).size())   # torch.Size([15, 10])
    # print(Y.view(-1).size())            # torch.Size([15])

    loss = criterion(output.view(-1, 10), Y.view(-1)) # prediction, correct
    loss.backward() # calc gradient
    optimizer.step()

    result = output.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.056679606437683 prediction:  [[7 9 1 0 5 6 1 8 2 3 4 1 0 5 6]] true Y:  [[7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]] prediction str:  if you want you
1 loss:  1.0561991930007935 prediction:  [[7 9 1 0 5 6 1 8 2 3 4 1 0 5 6]] true Y:  [[7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]] prediction str:  if you want you
2 loss:  1.0557199716567993 prediction:  [[7 9 1 0 5 6 1 8 2 3 4 1 0 5 6]] true Y:  [[7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]] prediction str:  if you want you
3 loss:  1.0552424192428589 prediction:  [[7 9 1 0 5 6 1 8 2 3 4 1 0 5 6]] true Y:  [[7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]] prediction str:  if you want you
4 loss:  1.0547661781311035 prediction:  [[7 9 1 0 5 6 1 8 2 3 4 1 0 5 6]] true Y:  [[7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]] prediction str:  if you want you
5 loss:  1.0542912483215332 prediction:  [[7 9 1 0 5 6 1 8 2 3 4 1 0 5 6]] true Y:  [[7, 9, 1, 0, 5, 6, 1, 8, 2, 3, 4, 1, 0, 5, 6]] prediction str:  if you want you
6 loss:  1.